In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data, DataLoader
import numpy as np
from graphdatascience import GraphDataScience
from py2neo import Graph
from neo4j import GraphDatabase
import networkx as nx
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd
import neo4jupyter
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling, remove_self_loops, add_self_loops, to_dense_adj, dense_to_sparse
from torch_geometric.nn import VGAE,GAE
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from torch_geometric.utils import to_dense_adj
#neo4jupyter.init_notebook_mode()
import tqdm as notebook_tqdm

In [3]:
import pickle
# Load the list of arrays from the .pkl file
with open('node_embeddings.pkl', 'rb') as pkl_file:
    extracted_data = pickle.load(pkl_file)

In [4]:
node_embeddings = extracted_data

In [6]:
diseases=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Disease.csv')
gene=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Gene.csv')
anatomy=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Anatomical.csv')
medication=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Medication.csv')
patient=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Patient.csv')
pathway=pd.read_csv('C:\\Users\\krjoob\\Desktop\\Knowledge graph\\UKBB\\UKB_CMR\\processing aftre extracting the data\\All_diseases\\nodes and edges\\Pathway.csv')

In [7]:
diseases_nodes=list(diseases.fname)
gene_nodes=list(gene.fname)


In [ ]:
diseases_embeddings = {key:node_embeddings[key] for key in diseases_nodes if key in node_embeddings}
gene_embeddings = {key:node_embeddings[key] for key in gene_nodes if key in node_embeddings}

### Gene-disease asscociation prediction AF

In [ ]:
All_Gene_nodes= pd.read_csv('C:\\Users\krjoob\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-afea5548-c3e4-4b02-b313-76660c29190a\\import\\Gene.csv')

In [10]:
list_all_gene = list(set(All_Gene_nodes.fname))

In [377]:
AF_gene_pos = ['GATA4','UBE4B','SCN10A','PPP2R3A','UST','AKAP6','LHX3','CEP68','LINC00477','WNT3','PPFIA4','PRRX1','WDR1','MBD5','IRF2BPL','SYNPO2L','MYPN',
 'TBX5','XPO1','HAND2','KCNN2','GOPC','TEX41','CASQ2','CAV2','NPPB','RPL3L','KCNN3','SSPN','SPATS2L','PITX2','NEURL1','GCOM1','VWF','LRIG1','KDM1B',
 'PHLDB2','PLN','TNFSF12','SLC9B1','NKX2-5','KIF3C','KCNJ8','RPS2','CAV1','ARHGAP10','HIP1R','HCN4','PKP2','IGF1R','ABCC9','SOX5','DNAH10','KCNE5',
 'LRMDA','SMAD7','SCMH1','TTN','HSF2','MYH6','ATXN1','CREB5','MYOCD','PSMB7','ARNT2','ASAH1','TUBA8','NR3C1','AGBL4','AOPEP','NACA','DGKB','MYOT',
 'SGCA','CYTH1','REEP1','CAND2','MYH7','SYNE2','GTF2I','SH3PXD2A','COG5','RBM20','MYO1C','HERC1','LINC01426','CASZ1','FBRSL1','WNT8A','MYOZ1','WIPF1',
 'CUL4A','CGA','SLC35F1','CDKN1A','SCN5A','USP3','C1orf185','XPO7','PHLDA1','KCNJ5','ZPBP2','GNB4','OPN1SW','XXYLT1','USP34','ZFHX3','MIR1-1HG','ARMH3',
 'FGF5','FRMD4B','SORL1','CYP2C19','NOS3','CDK6','HSPG2','LRRC10','LINC01142','HTR4','ERBB4','SLIT3','THRB','MIR6500','KCNA5','GOSR2','SLC27A6','SNRNP27',
 'KCNH2','KCNE2','NUCKS1','KCNJ2','PTK2','PAK2','SIRT1','DPF3','SOX15','ZNF462','SUN1','CFL2','SELE','CAMK2D','ACE','DES','REEP3','BEST3','CALU','FBXO32',
 'EPHA3','MYO18B','NEBL','MAPT','KCND3','GYPC','ESR2','EDN1','TTN-AS1','MIR30B','NAV2','MEX3C','GJA5']


In [530]:
AF_gene_neg =[
    'SPATA13', 'PRDM12', 'VWA8', 'PAGR1', 'S1PR2', 'CHAMP1', 'WFDC9', 'EP400P1', 'FAM131A', 'KLHL3',
    'AHSA1', 'KCNMB2', 'SLC25A31', 'SIKE1', 'DUOXA1', 'CHTF8', 'GSTT2', 'OSBPL7', 'PCF11', 'TLR7',
    'AAGAB', 'MGST1', 'MAMDC2', 'DNAL4', 'VAX2', 'IGKV4-1', 'CLP1', 'UNC79', 'CYP46A1', 'CRYGS',
    'CRACR2B', 'ZNF347', 'NR2F1', 'ADAMTS4', 'ISCU', 'FAM72A', 'AGPAT5', 'KIF3B', 'CABCOCO1', 'ARMC2',
    'SMIM2', 'CALCOCO1', 'ETHE1', 'HRC', 'PEX13', 'SH3BGR', 'NFASC', 'CYP27C1', 'MED12L', 'NBAS',
    'SOWAHC', 'ALDOB', 'ABTB1', 'TMIGD3', 'FREM1', 'BBIP1', 'PATE1', 'GMEB1', 'OPA3', 'TAS2R13', 'ABR',
    'C5orf34', 'ARMC12', 'RFLNA', 'TCN1', 'PDZD11', 'SWAP70', 'TEP1', 'COX6A2', 'HINT1', 'SCARNA27',
    'HINT3', 'TTC8', 'PSG2', 'DHRS9', 'ITFG2', 'HABP2', 'USB1', 'NSMCE1', 'TAS2R7', 'SLITRK4', 'CNPY1',
    'C9orf153', 'IGHV4-31', 'POC1B-GALNT4', 'METTL17', 'TUBD1', 'ESCO2', 'TMEM54', 'ANKRD53', 'FECH',
    'IGLV3-12', 'CORT', 'GPHA2', 'C12orf29', 'MED11', 'BTF3L4', 'STEAP4', 'MIR425', 'C16orf46', 'WNT3A',
    'C3orf35', 'TPTEP1', 'LAD1', 'SETD5', 'OR2H1', 
    'PAXX', 'GAGE5', 'GID4', 'ZNF350', 'PSORS1C3',
    'IGKV1D-39', 'RPS6KC1', 'NSA2', 'RBMY2BP','ROBO1', 'METTL27', 'SMIM19', 'ASCL4', 'HOATZ',
    'APCDD1L', 'KLHL21', 'SPATA1', 'FAM43A', 'GALNT6', 'GRSF1', 'ST14', 'DLGAP3', 'FGF23',
    'GPT', 'QRICH2', 'SACS', 'ADAMDEC1', 'ALDH5A1', 'IGLV3-19', 
    'UPP1', 'ABCB7', 'GLP2R', 'SELENOM', 'ESRRB', 'WASH6P', 'OPN1LW', 'KLHL10', 'IL17RD', 'HES5',
    'WFDC2', 'PCDH9', 'AK8', 'EMSY', 'ENAH', 'SERF1A', 'H2AZ2', 'PPFIBP2', 'RNF115', 'SLC15A3',
    'CATSPERD', 'SPTLC3', 'LRRC8E', 'PAPOLG', 'BCLAF3'
]

In [536]:
list_of_pos_neg = AF_gene_pos+AF_gene_neg

In [537]:
list_uknown_genes = [item for item in list_all_gene if item not in list_of_pos_neg]

In [538]:
len(list_uknown_genes)

18286

In [540]:
gene_embeddigns_pos = [np.concatenate((gene_embeddings[key],diseases_embeddings.get('atrial fibrillation'))) for key in AF_gene_pos if key in gene_embeddings]
gene_embeddigns_neg = [np.concatenate((gene_embeddings[key],diseases_embeddings.get('atrial fibrillation'))) for key in AF_gene_neg if key in gene_embeddings]
uknown_gene_embeddigns = [np.concatenate((gene_embeddings[key],diseases_embeddings.get('atrial fibrillation'))) for key in list_uknown_genes if key in gene_embeddings]
AF_embeddings = diseases_embeddings.get('atrial fibrillation')

In [541]:
len(gene_embeddigns_pos)

160

In [542]:
len(gene_embeddigns_neg)

160

In [543]:
diseases_embeddings.get('atrial fibrillation')

array([-0.14054246,  3.0569754 ,  0.5226926 ,  0.9300724 ,  1.0516058 ,
       -3.6237304 ,  8.274083  ,  0.4921572 ,  7.5255065 ,  0.81561816],
      dtype=float32)

In [544]:
gene_disease_pos = pd.DataFrame(np.array(gene_embeddigns_pos))
gene_disease_pos['Label'] = 1
gene_disease_neg = pd.DataFrame(np.array(gene_embeddigns_neg))
gene_disease_neg['Label'] = 0

In [546]:
gene_disease_pos_neg = pd.concat([gene_disease_pos,gene_disease_neg])

In [547]:
uknown_genes_names= [key for key in list_uknown_genes if key in gene_embeddings]
uknown_gene_embeddigns_df = pd.DataFrame(np.array(uknown_gene_embeddigns))

In [548]:
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
import random 

random.seed(123456)
np.random.seed(123456)
X = gene_disease_pos_neg.iloc[:,0:-1]
y = gene_disease_pos_neg.iloc[:,-1]
# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=20)


In [520]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
# Define the parameter grid
param_grid = {
    'C': [0.1, 1,5, 10, 50, 100, 200, 300, 500],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['rbf', 'linear']
}

# Initialize the classifier
random.seed(123)
svm = SVC(probability=True, random_state=42)

# Initialize GridSearchCV
random.seed(123)
grid_search = GridSearchCV(estimator=svm, param_grid=param_grid, cv=5, n_jobs=-1, scoring='accuracy', verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_svm = grid_search.best_estimator_

print("Best parameters found: ", best_params)

# Evaluate the best estimator
y_pred_svm = best_svm.predict(X_test)
y_pred_proba_svm = best_svm.predict_proba(X_test)[:, 1]
print("Accuracy:", accuracy_score(y_test, y_pred_svm))
print("ROC AUC:", roc_auc_score(y_test, y_pred_proba_svm))


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best parameters found:  {'C': 500, 'gamma': 0.1, 'kernel': 'rbf'}
Accuracy: 0.75
ROC AUC: 0.7761485826001955


In [525]:
svm_my_dict = {}
for i in range(0,uknown_gene_embeddigns_df.shape[0]):
    y_pred_svm_p = svm.predict(uknown_gene_embeddigns_df.iloc[i:i+1,:])
    y_pred_proba_svm_p = svm.predict_proba(uknown_gene_embeddigns_df.iloc[i:i+1,:])[:, 1]
    print(uknown_genes_names[i],y_pred_svm_p[0],y_pred_proba_svm_p[0])
    if y_pred_svm_p[0]>0:
        svm_my_dict[uknown_genes_names[i]] =y_pred_proba_svm_p[0]

pos_prd_prob_svm = dict(sorted(svm_my_dict.items(), key=lambda item: item[1],reverse=True))

TRAK1 0 0.31577291603227486
C1RL 0 0.31051886732102285
VAMP5 1 0.9999889131127581
TUBA3E 0 0.31654398181805776
NTSR1 0 0.29716315976408025
MTURN 0 0.3936404549337856
PHGDH 1 0.9246602836660018
HPGDS 0 0.2922345664597306
DUS2 0 0.347083426121064
CWC27 0 0.3887406178642624
CTBP1 1 0.9999999992409003
ADAM30 0 0.2986330373395745
FADD 1 0.9927043098928142
ASB13 0 0.4156669986068168
RNF26 0 0.3685821187934301
NR4A2 1 0.8514327487103522
RAD51C 0 0.5518299560051023
DLGAP1 1 0.5800870461972188
ACBD4 0 0.34969616017536476
MED27 0 0.39628666598562406
NAGS 0 0.2786543075219297
POGLUT2 0 0.299484676052408
DPP6 0 0.29260893529779186
KLHL42 1 0.9423024231897467
MASP1 0 0.4507107038343039
NFKB2 1 0.9878055539128159
SLC41A1 1 0.7953170905332102
TAF5L 1 0.6435874257078837
MIR372 0 0.280844081477721
NDUFAF2 0 0.3390553932410321
IGKC 1 0.9122513167466759
SLC26A1 0 0.3116328121965894
CD4 1 0.9924680886365503
E2F2 0 0.47483706161682
ATP2A3 1 0.9826795231795777
OR4N5 0 0.29238330390530415
ODF3L2 0 0.30268130

In [527]:
pos_prd_prob_svm

{'DAPK1': 0.9999999999999699,
 'SRC': 0.9999999999999699,
 'EP300': 0.9999999999999699,
 'COPS5': 0.9999999999999699,
 'GATA1': 0.9999999999999699,
 'H2AX': 0.9999999999999699,
 'HSPA8': 0.9999999999999699,
 'PDHA1': 0.9999999999999699,
 'CASP8': 0.9999999999999699,
 'HNF4A': 0.9999999999999699,
 'CDC5L': 0.9999999999999699,
 'VIM': 0.9999999999999699,
 'FOXP3': 0.9999999999999699,
 'HDAC3': 0.9999999999999699,
 'CEBPB': 0.9999999999999699,
 'H1-5': 0.9999999999999699,
 'NFKBIA': 0.9999999999999699,
 'RPL15': 0.9999999999999699,
 'ZNF263': 0.9999999999999699,
 'NFYB': 0.9999999999999699,
 'FOXA1': 0.9999999999999699,
 'MAP3K7': 0.9999999999999699,
 'AKT1': 0.9999999999999699,
 'UBL4A': 0.9999999999999699,
 'MID2': 0.9999999999999699,
 'MAPK8': 0.9999999999999699,
 'MDM2': 0.9999999999999699,
 'ILF3': 0.9999999999999699,
 'AP2M1': 0.9999999999999699,
 'CDC42': 0.9999999999999699,
 'LNX1': 0.9999999999999699,
 'ARMC3': 0.9999999999999699,
 'YWHAG': 0.9999999999999699,
 'CUL3': 0.99999999